About the Dataset:

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks whether the news article is real or fake:
           1: Fake news
           0: real News





Importing the Dependencies

In [10]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thanu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
# printing the stopwords in English
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Data Pre-processing

In [13]:
# loading the dataset to a pandas DataFrame
news_dataset = pd.read_csv('train.csv')

In [14]:
news_dataset.shape

(44898, 5)

In [15]:
# print the first 5 rows of the dataframe
news_dataset.head()

title  \
0   BREAKING: GOP Chairman Grassley Has Had Enoug...   
1   Failed GOP Candidates Remembered In Hilarious...   
2   Mike Pence’s New DC Neighbors Are HILARIOUSLY...   
3  California AG pledges to defend birth control ...   
4  AZ RANCHERS Living On US-Mexico Border Destroy...   

                                                text       subject  \
0  Donald Trump s White House is in chaos, and th...          News   
1  Now that Donald Trump is the presumptive GOP n...          News   
2  Mike Pence is a huge homophobe. He supports ex...          News   
3  SAN FRANCISCO (Reuters) - California Attorney ...  politicsNews   
4  Twisted reasoning is all that comes from Pelos...      politics   

               date  label  
0     July 21, 2017      1  
1       May 7, 2016      1  
2  December 3, 2016      1  
3  October 6, 2017       0  
4      Apr 25, 2017      1

In [16]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [17]:
# replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [18]:
# merging the news title and text
news_dataset['content'] = news_dataset['title']+' '+news_dataset['text']

In [19]:
print(news_dataset['content'])

0         BREAKING: GOP Chairman Grassley Has Had Enoug...
1         Failed GOP Candidates Remembered In Hilarious...
2         Mike Pence’s New DC Neighbors Are HILARIOUSLY...
3        California AG pledges to defend birth control ...
4        AZ RANCHERS Living On US-Mexico Border Destroy...
                               ...                        
44893    Nigeria says U.S. agrees delayed $593 million ...
44894    Boiler Room #62 – Fatal Illusions Tune in to t...
44895    ATHEISTS SUE GOVERNOR OF TEXAS Over Display on...
44896    Republican tax plan would deal financial hit t...
44897    U.N. refugee commissioner says Australia must ...
Name: content, Length: 44898, dtype: object


In [20]:
# separating the data & label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']


In [21]:
print(X)
print(Y)

                                                   title  \
0       BREAKING: GOP Chairman Grassley Has Had Enoug...   
1       Failed GOP Candidates Remembered In Hilarious...   
2       Mike Pence’s New DC Neighbors Are HILARIOUSLY...   
3      California AG pledges to defend birth control ...   
4      AZ RANCHERS Living On US-Mexico Border Destroy...   
...                                                  ...   
44893  Nigeria says U.S. agrees delayed $593 million ...   
44894                  Boiler Room #62 – Fatal Illusions   
44895  ATHEISTS SUE GOVERNOR OF TEXAS Over Display on...   
44896  Republican tax plan would deal financial hit t...   
44897  U.N. refugee commissioner says Australia must ...   

                                                    text          subject  \
0      Donald Trump s White House is in chaos, and th...             News   
1      Now that Donald Trump is the presumptive GOP n...             News   
2      Mike Pence is a huge homophobe. He suppor

Stemming:

Stemming is the process of reducing a word to its Root word

example:
actor, actress, acting --> act

In [22]:
port_stem = PorterStemmer()

In [23]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [24]:
print(news_dataset['content'])

0         BREAKING: GOP Chairman Grassley Has Had Enoug...
1         Failed GOP Candidates Remembered In Hilarious...
2         Mike Pence’s New DC Neighbors Are HILARIOUSLY...
3        California AG pledges to defend birth control ...
4        AZ RANCHERS Living On US-Mexico Border Destroy...
                               ...                        
44893    Nigeria says U.S. agrees delayed $593 million ...
44894    Boiler Room #62 – Fatal Illusions Tune in to t...
44895    ATHEISTS SUE GOVERNOR OF TEXAS Over Display on...
44896    Republican tax plan would deal financial hit t...
44897    U.N. refugee commissioner says Australia must ...
Name: content, Length: 44898, dtype: object


In [25]:
#separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [26]:
print(X)

[' BREAKING: GOP Chairman Grassley Has Had Enough, DEMANDS Trump Jr. Testimony Donald Trump s White House is in chaos, and they are trying to cover it up. Their Russia problems are mounting by the hour, and they refuse to acknowledge that there are problems surrounding all of this. To them, it s  fake news,  or a  hoax.  However, the facts bear things out differently, and it seems that there are now cracks in the Congressional public leadership.Chuck Grassley (R-Iowa), who heads the Senate Judiciary Committee, is fed up. He is now demanding that Donald Trump, Jr. and former 2016 Trump Campaign Manager Paul Manafort testify before his committee regarding the now infamous shady meeting between Donald Trump and the shady Russian lawyer who promised dirt on 2016 Democratic Presidential nominee Hillary Clinton. In fact, this information is due, well, NOW. This demand sends a few signals to team Trump   most notably that they should not fire Special Counsel Robert Mueller under any circumsta

In [27]:
print(Y)

[1 1 1 ... 1 0 0]


In [28]:
Y.shape

(44898,)

In [29]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [30]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9561245 stored elements and shape (44898, 122513)>
  Coords	Values
  (0, 2032)	0.06523976713549882
  (0, 8736)	0.01836656488878326
  (0, 9121)	0.06489775580929881
  (0, 11039)	0.08337225160369759
  (0, 11309)	0.034740394603362326
  (0, 11326)	0.020966755779001398
  (0, 12038)	0.13121334495759746
  (0, 12246)	0.05154023938814098
  (0, 12694)	0.024839530580311842
  (0, 12705)	0.03861915927506088
  (0, 13296)	0.13488697231724767
  (0, 13741)	0.014853820379487018
  (0, 14221)	0.01567520801193811
  (0, 16811)	0.06201648263777099
  (0, 16855)	0.06418825397414778
  (0, 17043)	0.019101899520841174
  (0, 17069)	0.025914723409318895
  (0, 17874)	0.02915200657804841
  (0, 18433)	0.09080434829045847
  (0, 18477)	0.0491402304199652
  (0, 20483)	0.04677936326844122
  (0, 21529)	0.06734478105302971
  (0, 22023)	0.01480514536166317
  (0, 22619)	0.027780275124669736
  (0, 23282)	0.06858950667718144
  :	:
  (44897, 113697)	0.04148345386195692

Splitting the dataset to training & test data

In [43]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

Training the Model: Logistic Regression

In [44]:
model = LogisticRegression()

In [45]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

accuracy score

In [34]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [35]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9925385600534551


In [36]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [37]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9908685968819599


Making a Predictive System

In [38]:
X_new = X_test[3]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[1]
The news is Fake


In [39]:
print(Y_test[3])

1
